In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import labrotation.file_handling as fh
import h5py
from time import time
import matplotlib.pyplot as plt
import numpy as np
import os
from labrotation import file_handling as fh
from copy import deepcopy
import pandas as pd
import labrotation.two_photon_session as tps
import seaborn as sns
import uuid  # for unique labeling of sessions and coupling arrays (mouse velocity, distance, ...) to sessions in dataframe 
from matplotlib import cm  # colormap
import datadoc_util

# If exists, load environmental variables from .env file

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

# Set up data documentation directory

In [ ]:
# assumption: inside the documentation folder, the subfolders carry the id of each mouse (not exact necessarily, but they 
# can be identified by the name of the subfolder). 
# Inside the subfolder xy (for mouse xy), xy_grouping.xlsx and xy_segmentation.xlsx can be found.
# xy_grouping.xlsx serves the purpose of finding the recordings belonging together, and has columns:
# folder, nd2, labview, lfp, face_cam_last, nikon_meta, experiment_type, day
# xy_segmentation.xlsx contains frame-by-frame (given by a set of disjoint intervals forming a cover for the whole recording) 
# classification of the events in the recording ("normal", seizure ("sz"), sd wave ("sd_wave") etc.). The columns:
# folder, interval_type, frame_begin, frame_end.

# TODO: write documentation on contents of xlsx files (what the columns are etc.)
if "DATA_DOCU_FOLDER" in env_dict.keys():
    docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    docu_folder = fh.open_dir("Choose folder containing folders for each mouse!")
print(f"Selected folder:\n\t{docu_folder}")

In [ ]:
if "documentation" in os.listdir(docu_folder):
    mouse_folder = os.path.join(docu_folder, "documentation")
else:
    mouse_folder = docu_folder
mouse_names = os.listdir(mouse_folder)
print(f"Mice detected:")
for mouse in mouse_names:
    print(f"\t{mouse}")

In [ ]:
mouse_folder

### Load matlab-2p

In [ ]:
if "MATLAB_2P_FOLDER" in env_dict.keys():
    matlab_2p_folder = env_dict["MATLAB_2P_FOLDER"]
else:
    matlab_2p_folder = fh.open_dir("Choose matlab-2p folder")
print(f"matlab-2p folder set to:\n\t{matlab_2p_folder}")

In [ ]:
df_seg_complete = pd.DataFrame(columns = ["nd2", "interval_type", "frame_begin", "frame_end"])
df_grouping_complete = pd.DataFrame(columns = ["folder", "nd2", "labview", "lfp", "face_cam_last", "nikon_meta", "experiment_type", "mouse_id", "day"])

for mouse_id in mouse_names:
    print(mouse_id)
    seg_fpath = os.path.join(mouse_folder, mouse_id, mouse_id + '_segmentation.xlsx')
    grouping_fpath = os.path.join(mouse_folder, mouse_id, mouse_id + '_grouping.xlsx')
    if os.path.exists(seg_fpath) and os.path.exists(grouping_fpath):
        df_seg = pd.read_excel(seg_fpath)
        df_grouping = pd.read_excel(grouping_fpath)
        df_grouping["mouse_id"] = mouse_id
        # select only tmev, chr2_szsd, chr2_sd, chr2_ctl experiment data first
        df_grouping = df_grouping[df_grouping["experiment_type"].isin(["tmev", "tmev_ctl", "chr2_sd", "chr2_szsd", "chr2_ctl"])]
        # merge into large dataframes
        # print(f"\tseg bef: {len(df_seg_complete['nd2'])}")
        df_seg_complete = pd.concat([df_seg_complete, df_seg])
        # print(f"\tseg aft: {len(df_seg_complete['nd2'])}")
        # print(f"\tgro bef: {len(df_grouping_complete['nd2'])}")
        df_grouping_complete = pd.concat([df_grouping_complete, df_grouping])
        # print(f"\tgro aft: {len(df_grouping_complete['nd2'])}")
    else:
        print(f"Check if you set the correct folder (folder containing all subfolders with mouse names):")
        if not os.path.exists(seg_fpath):
            print(f"\t{seg_fpath} not found")
        if not os.path.exists(grouping_fpath):
            print(f"\t{grouping_fpath} not found")
        